>> #### Load required libraries

In [2]:
import tensorflow as tf
import librosa   #for audio processing
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile #for audio processing
import warnings
warnings.filterwarnings("ignore")

>> #### Load scripts

In [ ]:
from loader import data_loader
from preprocessing import Preprocessor
from models import Model

In [32]:
FILE_PATH = '../data/AMHARIC/data'
SAMPLE_RATE = 44100

In [ ]:
loader = data_loader(FILE_PATH)
preprocess = Preprocessor()
model = Model()

In [ ]:
audios = loader.load_audios()
audios = preprocess.preprocess_audios(audios)


In [ ]:
audios = loader.get_all_audios()


In [3]:
SAMPLE_RATE = 44100

In [34]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Conv2D, Dense, LSTM, MaxPooling2D, Bidirectional, Flatten

2022-06-07 15:07:28.733894: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-06-07 15:07:28.787673: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-07 15:07:28.787695: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [37]:
input_img = layers.Input(
    shape=(400, 600, 4), name="image", dtype="float32"
)
labels = layers.Input(name="label", shape=(None,), dtype="float32")

x = layers.Conv2D(
    32,
    (3, 3),
    activation="relu",
    kernel_initializer="he_normal",
    padding="same",
    name="Conv1",
)(input_img)
x = layers.MaxPooling2D((2, 2), name="pool1")(x)

x = layers.Conv2D(
    64,
    (3, 3),
    activation="relu",
    kernel_initializer="he_normal",
    padding="same",
    name="Conv2",
)(x)
x = layers.MaxPooling2D((2, 2), name="pool2")(x)
print(x.type_spec.shape)

new_shape = (x.type_spec.shape[-3], x.type_spec.shape[-2]*x.type_spec.shape[-1])
x = layers.Reshape(target_shape=new_shape, name="reshape")(x)
x = layers.Dense(64, activation="relu", name="dense1")(x)
x = layers.Dropout(0.3)(x)
x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=0.25))(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, dropout=0.25))(x)

x = layers.Dense(
    len(char_encoder.classes_) + 1, activation="softmax", name="dense2"
)(x)

output = CTC(name="ctc_loss")(labels, x)

# Model Definition
model = keras.models.Model(
    inputs=[input_img, labels], outputs=output, name="mod"
)
# add Optimizer
opt = tf.keras.optimizers.Adam()
# Compile the model and return
model.compile(optimizer=opt)
model.summary()

2022-06-07 15:08:02.698149: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-07 15:08:02.698176: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-07 15:08:02.698196: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jedi): /proc/driver/nvidia/version does not exist
2022-06-07 15:08:02.698617: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(None, 100, 150, 64)
Model: "ocr_model_v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, 400, 600, 4  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 400, 600, 32  1184        ['image[0][0]']                  
                                )                                                                 
                                                                                                  
 pool1 (MaxPooling2D)           (None, 200, 300, 32  0           ['Conv1[0][0]']                  
                                )                                 

In [38]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
print(X_train.shape, np.array(y_train).shape)
model.fit([X_train, np.array(y_train)], batch_size = 4, epochs = 100)

(100, 400, 600, 4) (100, 20)
Epoch 1/100
25/25 [==============================] - 178s 7s/step - loss: 179.5235
Epoch 2/100
25/25 [==============================] - 15s 591ms/step - loss: 79.7463
Epoch 3/100
25/25 [==============================] - 9s 362ms/step - loss: 77.2045
Epoch 4/100
25/25 [==============================] - 9s 352ms/step - loss: 76.8000
Epoch 5/100
25/25 [==============================] - 8s 314ms/step - loss: 76.5132
Epoch 6/100
25/25 [==============================] - 8s 309ms/step - loss: 76.2412
Epoch 7/100
25/25 [==============================] - 8s 317ms/step - loss: 75.9082
Epoch 8/100
25/25 [==============================] - 9s 348ms/step - loss: 75.6581
Epoch 9/100
25/25 [==============================] - 9s 348ms/step - loss: 75.4062
Epoch 10/100
25/25 [==============================] - 8s 315ms/step - loss: 75.0994
Epoch 11/100
25/25 [==============================] - 8s 299ms/step - loss: 75.0069
Epoch 12/100
25/25 [==============================] - 8